In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
sns.set_style("darkgrid")


Merging two datasets into one master dataset. 

In [9]:
master_df = pd.DataFrame()

In [10]:
for file in os.listdir(os.getcwd()):
    if file.endswith('.csv'):
        master_df = master_df.append(pd.read_csv(file))

/var/folders/lz/66fc_c917c5gz3z1hh24w_l40000gn/T/ipykernel_36106/2531444959.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_df = master_df.append(pd.read_csv(file))
/var/folders/lz/66fc_c917c5gz3z1hh24w_l40000gn/T/ipykernel_36106/2531444959.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_df = master_df.append(pd.read_csv(file))
/var/folders/lz/66fc_c917c5gz3z1hh24w_l40000gn/T/ipykernel_36106/2531444959.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_df = master_df.append(pd.read_csv(file))


In [11]:
master_df.to_csv('masterDataSet.csv',index=False)

In [13]:
data = pd.read_csv("masterDataSet.csv")
len(data)
data.head(10)

,Unnamed: 0,Sentence_id,New_Sentence,Type
0,0,GERRES15609,Author and/or Review architecture/design and o...,Responsibility
1,1,PHERES15784,Should be able to develop custom dynamic shape...,Responsibility
2,2,GERREQ10457,Experience in working crosslly with a larger ...,Requirement
3,3,GERSKL27235,"Previous business experience, including but no...",Skill
4,4,HONSSK18415,Delivering fast and right the first time.,SoftSkill
5,5,INDRES9494,Track department expenses and capital budget .,Responsibility
6,6,HONRES51852,Meet performance metrics.,Responsibility
7,7,UAEREQ34683,YOU MUST HAVEBachelors degree in Engineering.,Requirement
8,8,UAEREQ4612,After COVID-19: Ability to travel to manufactu...,Requirement
9,9,INDEDU42721,Six Sigma qualification to at least Green Bel...,Education
